In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn
import csv

In [2]:
TARGET_DIR = "./runs/1560327157/"
POSITIVE_DATA_FILE = "./data/rt-polaritydata/rt-polarity.pos"
NEGATIVE_DATA_FILE = "./data/rt-polaritydata/rt-polarity.neg"
BATCH_SIZE = 1024

x_raw, y_test = data_helpers.load_data_and_labels(POSITIVE_DATA_FILE, NEGATIVE_DATA_FILE)
x_raw.append("hello world")
y_test =  np.row_stack((y_test,(0,1)))
y_test = np.argmax(y_test, axis=1)

# Map data into vocabulary
vocab_path = os.path.join(TARGET_DIR, "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

Instructions for updating:
Please use tensorflow/transform or tf.data.


In [3]:
print("\nEvaluating...\n")

checkpoint_file = tf.train.latest_checkpoint(os.path.join(TARGET_DIR,"checkpoints"))

with tf.Session() as sess:
     # Load the saved meta graph and restore variables
    saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
    saver.restore(sess, checkpoint_file)

    # Get the placeholders from the graph by name
    input_x = sess.graph.get_tensor_by_name("input_x:0")
    
    dropout_keep_prob = sess.graph.get_tensor_by_name("dropout_keep_prob:0")

    # Tensors we want to evaluate
    predictions = sess.graph.get_tensor_by_name("output/predictions:0")

    # Generate batches for one epoch
    batches = data_helpers.batch_iter(list(x_test), BATCH_SIZE, 1, shuffle=False)

    # Collect the predictions here
    all_predictions = []

    for x_test_batch in batches:
        batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
        all_predictions = np.concatenate([all_predictions, batch_predictions])

#Print accuracy if y_test is defined
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

# Save the evaluation to a csv
predictions_human_readable = np.column_stack((np.array(x_raw), all_predictions,y_test))
out_path = os.path.join(TARGET_DIR, "prediction.csv")
print("Saving evaluation to {0}".format(out_path))
with open(out_path, 'w') as f:
    csv.writer(f).writerows((("Sentence","Prediction","Label"),))
    csv.writer(f).writerows(predictions_human_readable)


Evaluating...

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from F:\JupyterRoot\DL_Parctise\tensorflow_study\14.cnn text classification\runs\1560327157\checkpoints\model-1400
num_batches_per_epoch: 11
Total number of test examples: 10663
Accuracy: 0.973084
Saving evaluation to ./runs/1560327157/prediction.csv
